**Final Project: Reading Emails **bold text** </br>
The program gets required credentials to access gmail(jackline.w.gathoni.24@dartmouth.edu) and returns extracted details from the email which include the subject, sender, recipient and body. </br>
Dartmouth College, LING48, Spring 2023 </br>
Jackline Gathoni Wambui (jackline.w.gathoni.24@dartmouth.edu) </br>
Dahlia Igiraneza (dahlia.igiraneza.24@dartmouth.edu) </br>
Paige Nakai (paige.m.nakai.24@dartmouth.edu) </br>



In [1]:
#install necessary packages
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
#Get credentials from gmail API(jackline.w.gathoni.24@dartmouth.edu)
#Code from https://developers.google.com/gmail/api/quickstart/python 
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']


def getService():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('gmail_token.json'):
        creds = Credentials.from_authorized_user_file('gmail_token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('gmail_token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        return service

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')



In [3]:
# Reads incoming emails to check if theyre related to shift-dropping or not and then parse for relevant info(date, time and location).
import base64
def getEmails(service):
    email_details={}
    # to read new messages, get unread specifically,  # add check for no new messages if you decide to do unread messages only
    # results = service.users().messages().list(userId="me", labelIds=["INBOX"], q="from:specific email, is:unread").execute()
    results = service.users().messages().list(maxResults=1, userId='me', labelIds=["INBOX"]).execute()

   
    messages = results.get('messages')  # messages is a list of dictionaries where each dictionary contains a message id.
  
   
    # iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
  
        # Use try-except to avoid any Errors
        try:
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']
            parts = payload.get('parts')
         
  
            # Look for Subject, Sender Email and other Recipient emails in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                if d['name'] == 'To':
                    recipient = d['value']
                
  
            # from stacksoiverflow to decode the body into plain text
            for part in parts:
                if part["mimeType"] in ["text/plain"]:
                    data = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
                   

            #add parts of the email to the email dictionary to be used in sending an automated response when available
            email_details["Subject"]=subject
            email_details["Sender"]= sender
            email_details["Message"]= subject +' \n' + data  #adds the subject to the body of the email in case the subject contains relevant information
            email_details["Recipients"]= recipient
            
        except:
            pass
    return email_details
  
  

In [4]:
service =getService()
getEmails(service)

{'Subject': 'CPT',
 'Sender': '"Keriann K. Niles" <Keriann.K.Niles@dartmouth.edu>'}